# Generating PLUMED input files from Python

#### Motivation:
The *setup process for a PLUMED project can be long and involved* my new users. For each atom group and collective variable, the user must manually pull atom numbers from a configuration file. Even for experts, complicated collective variables involving many unique virtual atoms or atom groups may require extensive consultation with the user manual and inflexible, hand-written automation scripts. Moving this process to **Python** will significantly **decrease the barrier to entry** for enhanced sampling simulations, **expedite** the setup process, and **enable** input sharing in the field.

#### Strategy: 
I have added a `generate_input()` function to plumitas which takes an input configuration for your system and a dictionary which describes your desired enhanced sampling simulation. The topology is handled by `mdtraj` so that simple VMD-style selections can be converted to atom numbers for the final plumed.dat file. Requiring manual entry into the python dictionary isn't much better than manually hunting down atom numbers, so there is room for improvement... I envision this improvement coming in the form of an **interactive GUI** which will prevent erroneous user input and generate the input dictionary without free form text entry. The following procedure will then quickly get users from configuration to simulation.

#### Procedure:
    1. import mdtraj and plumitas

In [1]:
import mdtraj as md

import plumitas as plm

    2. load configuration into python w/ mdtraj

In [2]:
top = 'data/topology/conf.gro'
traj = md.load(top)
topology = traj.top

    3. build python dict/class describing enhanced sampling* 
                     *done w/ interactive GUI in the future!

In [3]:
plumed = {'header': {'restart': False,
                     'wholemolecules': ['protein', 'resname ALA']},

          'groups': {'sidechain_com': {'com': 'sidechain and resname ALA'}
                     },

          'collective_variables': {
              'phi': {'torsion': {'angle': 'phi',
                                  'resid': 2,
                                  'atoms': '',
                                  }
                      },
              'psi': {'torsion': {'atoms': '7,9,15,17',
                                  'angle': 'psi',
                                  'resid': 2,
                                  },
                      },
          },

          'bias': {'pbmetad': {'label': 'pbmetad',
                               'arg': 'phi,psi',
                               'temp': '300',
                               'pace': '500',
                               'biasfactor': '15',
                               'height': '1.2',
                               'sigma': '0.35,0.35',
                               'grid_min': '-pi,-pi',
                               'grid_max': 'pi,pi',
                               'grid_spacing': '0.1,0.1',
                               'file': 'HILLS_phi,HILLS_psi'
                               }
                   },

          'footer': {'print': {'stride': '500',
                               'arg': 'phi,psi,pbmetad.bias',
                               'file': 'COLVAR'
                               },
                     }
          }


    4. generate your PLUMED runfile

In [4]:
plm.generate_input(topology, **plumed)

### That's it!
We can check out our new plumed.dat file below.

In [5]:
with open('plumed.dat') as new_file:
    for line in new_file:
        print(line)

WHOLEMOLECULES ENTITY0=0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21 ENTITY1=6,7,8,9,10,11,12,13,14,15 



sidechain_com: COM ATOMS=7,9,10,11,12,13



phi: TORSION 4,6,8,14

psi: TORSION 7,9,15,17



PBMETAD ...

LABEL=pbmetad

ARG=phi,psi

TEMP=300

PACE=500

BIASFACTOR=15

HEIGHT=1.2

SIGMA=0.35,0.35

GRID_MIN=-pi,-pi

GRID_MAX=pi,pi

GRID_SPACING=0.1,0.1

FILE=HILLS_phi,HILLS_psi

... PBMETAD



PRINT STRIDE=500 ARG=phi,psi,pbmetad.bias FILE=COLVAR 



